In [23]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import json
from matplotlib.backends.backend_pdf import PdfPages
import datetime
import time
import jsonlines
from PIL import Image
from scipy.optimize import curve_fit
import scipy as sy
from annotation_dset import annotationDatabase
import pickle as pkl

## useful constants
ANNOTATIONS_DIR = '../data/annotation-dataset-stylegan2/annotations' # path to many datasets, includes on directory before the below dirs
LABELS_FILE = '/annotation-tool/data.json'
OUT_MANIFEST_FILE = '/manifests/output/output.manifest'
WORKERS_RESPONSE = '/annotations/worker-response'
IMAGE_PATH = '../data/annotation-dataset-stylegan2/images' # location of images
OUTPUT_PDF_DIR = './figures' # location of output pdfs
SEQUENCE_LENGTH = 10 # number of images per sequence

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Get list of experiments in annotation dir. Alternatively, specify ones you care about.
annotations_dict = {}
annotations_labels_dict = {}
experiments = [os.path.split(f)[1] for f in glob.glob(os.path.join(ANNOTATIONS_DIR, '*'))]
print('experiments', experiments)
# experiments = ['calibration-random-v2-facial-hair-000'] #'perona-causal-faces-uncanny-000']

# If output directory doesn't exist, make it
if not os.path.exists(OUTPUT_PDF_DIR):
    os.makedirs(OUTPUT_PDF_DIR)

# Do analysis figures for each experiment
for EXPERIMENT_LABEL in experiments:
    ANNOTATIONS_PATH = f'{ANNOTATIONS_DIR}/{EXPERIMENT_LABEL}{WORKERS_RESPONSE}'
    OUT_MANIFEST_PATH = f'{ANNOTATIONS_DIR}/{EXPERIMENT_LABEL}{OUT_MANIFEST_FILE}'
    LABELS_PATH = f'{ANNOTATIONS_DIR}/{EXPERIMENT_LABEL}{LABELS_FILE}'
    OUT_PDF_FILE_NAME = os.path.join(OUTPUT_PDF_DIR, EXPERIMENT_LABEL + '.pdf')

    #Read labels using annotation-tool/data.json and assign integers to labels 
    with open(LABELS_PATH, 'r') as labels_file:
        labels_data = json.load(labels_file)['labels']
        LABELS = [l['label'] for l in labels_data]
    labelScores = {l:i for (i, l) in enumerate(LABELS)} # generate numerical scores for the labels - useful in regression

    # Get ordered list of image names from output manifest
    image_names = []
    with jsonlines.open(OUT_MANIFEST_PATH) as reader:
        for obj in reader:
            _, name = os.path.split(obj['source-ref']) #remove leading path
            image_names.append(name)

    # Make annotation file name list in proper order
    annotation_file_names = []
    for i in range(len(image_names)):
        annotation_file_names += glob.glob(ANNOTATIONS_PATH + '/*/%d/*.json'%i)
    
    # put together the database of the annotator IDs and their work
    annotations = annotationDatabase(annotation_file_names, labelScores)
    annotations.startPDFReport(OUT_PDF_FILE_NAME, EXPERIMENT_LABEL)
    annotations.displayAnnotatorsWork()
    annotations.displaySequenceAnnotations(image_names, LABELS, SEQUENCE_LENGTH, IMAGE_PATH)
    annotations.endPDFReport()
    
    # print(labelScores)
    print(LABELS, np.array(annotations.imageScores).shape)
    annotations_dict[EXPERIMENT_LABEL] = annotations.imageScores
    annotations_labels_dict[EXPERIMENT_LABEL] = LABELS

pkl.dump(annotations_dict, open('01_annotations_dict.pkl', 'wb'))
pkl.dump(annotations_labels_dict, open('01_annotations_labels_dict.pkl', 'wb'))